# Нейросеть для автодополнения текстов

<div class='alert alert-info'> 

Финальное задание спринта 2. 

Задача – создать нейросеть, которая на основе начала фразы предсказывает её продолжение. 

Последовательность работы: 

1. Взять датасет, очистить его, подготовить для обучения модели.
2. Реализовать и обучить модель на основе рекуррентных нейронных сетей.
3. Замерить качество разработанной и обученной модели.
4. Взять более «тяжёлую» предобученную модель из Transformers и замерить её качество.
5. Проанализировать результаты и дать рекомендации разработчикам: стоит ли использовать лёгкую модель или лучше постараться поработать с ограничениями по памяти и использовать большую предобученную.

</div>

## Загрузка библиотек, установка констант

In [1]:
import os
# import random
import re

import numpy as np
import pandas as pd
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer

c:\Users\User\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# traditionally, 
SEED = 42
# batch size
BATCH_SIZE = 128
# Train mode; if 'preliminar', just verify the code, 
# if 'final' – train the ultimate models versions. 
# When 'preliminar' mode is switched on, reduce the
# data volume to 5_000 tweets. 
TRAIN_MODE = 'preliminar' 
# name of transformer model
MODEL_NAME = 'distilgpt2'

## Обработка данных

<div class='alert alert-info'>

Данные нужно 

- привести к нижнему регистру;
- удалить ссылки, упоминания, эмодзи (по необходимости);
- заменить нестандартные символы;
- токенизировать текст.

</div>

### Загрузка данных

In [3]:
# set data directory
os.chdir('C:/Users/User/Yandex.Disk/DS.projects/LSTM.Praktikum/data')
# read raw text data and save to array
with open('raw_data.txt', 'r', encoding='utf-8') as file:
    raw_data = np.array(file.read().lower().splitlines())

if TRAIN_MODE == 'preliminar': 
    raw_data = raw_data[:5_000]

### Очистка данных

In [4]:
# define the function for clearing and splitting of data
def split_and_clean(row): 
    # remove the mentions (@*)
    row = re.sub(r'@.*?\s', '', row)
    row = re.sub(r'@.*?\Z', '', row)

    # remove the URLs (http or www)
    row = re.sub(r'www.*?\s', '', row)
    row = re.sub(r'http.*?\s', '', row)
    row = re.sub(r'www.*?\Z', '', row)
    row = re.sub(r'http.*?\Z', '', row)

    # remove emojies ('*...anything')
    row = re.sub(r'\*([^ ]+)\s', '', row)
    row = re.sub(r'\*([^ ]+)\Z', '', row)

    # remove special symbols (&*;)
    row = re.sub(r'&([^ ]+)\;', '', row)

    # remove everything except of letters and numbers
    row = re.sub(r'[^a-z0-9\s]', '', row)

    # substitute the multiple spaces to single ones
    row = re.sub(r'[\s+]', ' ', row)

    # split the strings by spaces
    row = row.split(' ')

    # remove the empty elements from lists
    row = list(filter(None, row))
    
    return(row)

raw_data = list(map(split_and_clean, raw_data))

In [5]:
# check the result of raw data import (deliberabely without seed)
if TRAIN_MODE == 'preliminar': 
    for _ in np.random.randint(0, len(raw_data), 10): 
        print(raw_data[_])

['will', 'have', 'a', 'meeting', 'in', 'an', 'hour', 'to', 'explain', 'which', 'version', 'of', 'oaw', 'we', 'use', 'what', 'to', 'say', 'none', 'atm', 'its', 'just', 'a', 'heap', 'of', 'unbundled', 'emf', 'tools']
['im', 'in', 'pain']
['i', 'dont', 'think', 'there', 'is', 'any', 'kind', 'of', 'good', 'stroke', 'ill', 'wait', 'to', 'hear', 'from', 'you', 'i', 'love', 'that', 'little', 'cat', 'l', 'xxx']
['just', 'realised', 'how', 'gutted', 'she', 'is', 'to', 'return', 'back', 'to', 'london', 'without', 'the', 'end']
['seems', 'jruby', 'support', 'for', 'hpricot', 'is', 'now', 'two', 'versions', 'behind']
['sad', 'that', 'the', 'time', 'shift', 'means', 'its', 'dark', 'when', 'we', 'go', 'home']
['no', 'i', 'lost', 'a', 'loyal']
['still', 'feeling', 'almost', 'entirely', 'overwhelmed', 'by', 'an', 'uncomfortable', 'desire', 'for', 'swift', 'and', 'violent', 'revenge']
['i', 'just', 'cant', 'commit', 'the', 'time', 'though', 'my', 'play', 'time', 'isnt', 'the', 'same', 'as', 'everyone',

<div class='alert alert-info'>

Поскольку по условию задания модель получает на вход 3/4 исходного текста, фразы, в которых осталось три слова и менее, следует удалить. Более того, из-за использования `ROUGE-2` как одной из метрик, удалять надо тексты, в которых четвёртая часть равна двум словам, т.е., общая длина не менее шести. 

Итого: вычищаем из корпуса текстов все последовательности короче шести слов.

</div>

In [6]:
print(f'Количество фраз до удаления слишком коротких: {len(raw_data)}.')
# drop too short phrases
raw_data = [phrase for phrase in raw_data if len(phrase) > 5]

# check the results
print(f'Количество фраз после удаления слишком коротких: {len(raw_data)}.')
if len(min(raw_data, key=len)) < 6: 
    print('Очистка от коротких фраз прошла с ошибкой.')
else: 
    print('Все короткие фразы удалены.')


Количество фраз до удаления слишком коротких: 5000.
Количество фраз после удаления слишком коротких: 4239.
Все короткие фразы удалены.


In [7]:
# save the processed (cleaned) dataset
with open('processed_data.txt', 'w+', encoding='utf-8') as file: 
    for row in raw_data:
        file.write(' '.join(row) + '\n')

### Разбиение данных

<div class='alert alert-info'>

По условию задания, обучающая выборка 80%, валидационная и тестовая по 10%.

</div>

In [8]:
# create train, test and valid datasets
train, interhim = train_test_split(raw_data, train_size=0.8, random_state=SEED)
valid, test = train_test_split(interhim, train_size=0.5, random_state=SEED)

del interhim

# check splitting
print(f'В обучающей выборке содержится {len(train)} фраз.')
print(f'В валидационной выборке содержится {len(valid)} фраз.')
print(f'В тестовой выборке содержится {len(test)} фраз.')

В обучающей выборке содержится 3391 фраз.
В валидационной выборке содержится 424 фраз.
В тестовой выборке содержится 424 фраз.


In [9]:
# save the datasets to disk
with open('train.txt', 'w+', encoding='utf-8') as file: 
    for row in train:
        file.write(' '.join(row) + '\n')
with open('valid.txt', 'w+', encoding='utf-8') as file: 
    for row in valid:
        file.write(' '.join(row) + '\n')
with open('test.txt', 'w+', encoding='utf-8') as file: 
    for row in test:
        file.write(' '.join(row) + '\n')

## Подготовка к токенизации данных и их загрузке в модель

In [10]:
# add pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
if TRAIN_MODE == 'preliminar': 
    print(tokenizer)

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)


In [11]:
# check the results of tokenization
if TRAIN_MODE == 'preliminar': 
    print(tokenizer.encode(
        raw_data[11], 
        is_split_into_words=True, 
        add_special_tokens=True, 
        return_tensors='pt')
        )
    print(len(tokenizer.encode(
        raw_data[11], 
        is_split_into_words=True, 
        add_special_tokens=True, 
        return_tensors='pt')[0])
        )
    print(tokenizer.tokenize(
        raw_data[11], is_split_into_words=True, return_tensors='pt')
        )
    print(len(tokenizer.tokenize(
        raw_data[11], is_split_into_words=True, return_tensors='pt'))
        )
    print(raw_data[11])
    print(len(raw_data[11]))

tensor([[  289,   692,   271,  1918,  3715,   481,  5938,   502, 15052,   284,
          2342,   319,  2646,   266,   563,   318, 13445,  2005,   407,   503,
           783]])
21
['Ġh', 'oll', 'is', 'Ġdeath', 'Ġscene', 'Ġwill', 'Ġhurt', 'Ġme', 'Ġseverely', 'Ġto', 'Ġwatch', 'Ġon', 'Ġfilm', 'Ġw', 'ry', 'Ġis', 'Ġdirectors', 'Ġcut', 'Ġnot', 'Ġout', 'Ġnow']
21
['hollis', 'death', 'scene', 'will', 'hurt', 'me', 'severely', 'to', 'watch', 'on', 'film', 'wry', 'is', 'directors', 'cut', 'not', 'out', 'now']
18


In [12]:
def tokenize(row):
    return tokenizer.encode(
        row, is_split_into_words=True, add_special_tokens=True, return_tensors='pt', 
        )
if TRAIN_MODE == 'preliminar': 
    print(tokenize(train[np.random.randint(0, len(train), 1).item()]))

tensor([[  703,  1312,   285,   824,   262, 39442,  5494,    86,   912,   340,
           286,   743,  1312,  1265,    72, 18869,  3285]])


In [13]:
# класс датасета
class MaskedDataset(Dataset):
    def __init__(self, texts, tokenizer, target_mode='single'):
        # the list for pairs, including the start of tokenized text and their end
        self.samples = []

        for line in texts: 
            # tokenize the text
            token_ids = tokenize(line) 
            # create a context (the known 75% of tokens)
            context = token_ids[0][0:(3 * len(token_ids[0]) // 4)] 
            if target_mode == 'complete': 
                # create a target (the last 25% of tokens which must be reconstructed)
                target = token_ids[0][(3 * len(token_ids[0]) // 4):] 
            elif target_mode == 'single': 
                # create a target (the single token following to first 75% of tokens)
                target = token_ids[0][(3 * len(token_ids[0]) // 4) ]
            # join the 'context' and 'target' as tulpe and add to 'samples'
            self.samples.append((context, target))
           
    def __len__(self):
        # return the length of samples
        return len(self.samples) 

    def __getitem__(self, idx):
        # return the context and target with given number ('idx')
        x, y = self.samples[idx] 
        return {
            'context': x.detach().clone(), 
            'target': y.detach().clone()
        }

In [14]:
if TRAIN_MODE == 'preliminar': 
    print(MaskedDataset(train, tokenize)[np.random.randint(0, len(train), 1).item()])

{'context': tensor([  616, 12385,   626, 30662, 26197,   257,  1643,   618,  1312,   373,
        10833,   616,  3290,   656,   262,  3996,   286,   616,  7779,   616,
         8046,  1312]), 'target': tensor(760)}


In [15]:
# create tokenized datasets
train_tok = MaskedDataset(train, tokenize)
valid_tok = MaskedDataset(valid, tokenize)
test_tok = MaskedDataset(test, tokenize)

In [16]:
def collate_fn(batch): 
    # список текстов и классов из батча
    contexts = [item['context'] for item in batch]
    targets = torch.stack([item['target'] for item in batch])

    # дополняем тексты в батче padding'ом
    padded_contexts = pad_sequence(contexts, batch_first=True, padding_value=0)

    # lengths = [len(text) for text in texts]
    lengths = torch.tensor([len(text) for text in contexts])
    # считаем маски
    masks = (padded_contexts != 0).long()

    # возвращаем преобразованный батч
    return padded_contexts, masks, lengths, targets


In [17]:
# create dataloaders
train_dataloader = DataLoader(
    train_tok, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
    )
valid_dataloader = DataLoader(
    valid_tok, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
    )
test_dataloader = DataLoader(
    test_tok, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
    )

## Рекуррентная сеть

<div class='alert alert-info'>

Условия выполнения этапа: 

- Напишите код модели на основе LSTM. 
- В методе `forward` модель должна принимать на вход последовательность токенов и предсказывать следующий токен.
- Дополнительно для модели реализуйте метод генерации нескольких токенов.

</div>

<div class='alert alert-info'>

Последовательность работы модели: 

1. Модель получает на вход начальную последовательность токенов *X*.
2. Затем она предсказывает вероятности следующего токена *P*(*w*<sub>n+1</sub>).
3. Токен *w*<sub>n+1</sub>, имеющий наибольшую вероятность, добавляется к последовательности. 
4. Модель снова делает предсказание *P*(*w*<sub>n+2</sub>).
5. Процесс повторяется, пока не выполнится одно из условий:
    - сгенерирован токен окончания (например, `<eos>`)
    - или достигнута максимальная длина генерации.

</div>

In [34]:

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, hidden_dim=256):
        super().__init__()

        # embedding layer
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, bidirectional=True)

        # out_dim for sum
        out_dim = hidden_dim 

        # output linear layer
        self.fc = nn.Linear(out_dim, vocab_size)

    def forward(self, x, lengths): 
        # embed the text
        emb = self.embedding(x) 
        pack = pack_padded_sequence(
            emb, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
        # get the output of recurrent layer ('out')
        out, _ = self.rnn(pack) 
        out, _ = pad_packed_sequence(out)

        # скрытые состояния <MSAK> токена 
        # после двух проходов двунаправленной сети
        hidden_forward = out[:, :, :out.size(2)//2]
        hidden_backward = out[:, :, out.size(2)//2:]

        # агрегация скрытых состояний в зависимости от self.combine
        hidden_agg = hidden_forward + hidden_backward

        linear_out = self.fc(hidden_agg)

        return linear_out
    



In [35]:
model_lstm = LSTMClassifier(vocab_size=tokenizer.vocab_size)
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()
metric_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)


In [36]:
# функция замера лосса и accuracy for the single output token
def evaluate_single_token(model, loader):
    model.eval()
    correct, total = 0, 0
    sum_loss = 0
    with torch.no_grad():
        for x_batch, mask_batch, len_batch, y_batch in loader:
            x_output = model.forward(x_batch) # выход модели для входа x_batch
            loss = criterion(x_output, y_batch) # функция потерь
            preds = torch.argmax(x_output, dim=1) # предсказанные токены
            accuracy += (preds == y_batch).sum().item() # количество верно угаданных токенов
            total_batch_size += y_batch.size(0) # размер батча
            sum_loss += loss.item() # суммарная функция потерь
    
    # лосс и accuracy
    avg_loss = sum_loss / len(loader)
    accuracy = correct / total_batch_size
    return avg_loss, accuracy


In [37]:
# Основной цикл обучения
n_epochs = 3

for epoch in range(n_epochs):
    model_lstm.train()
    train_loss = 0.
    for x_batch, mask_batch, len_batch, y_batch in tqdm(train_dataloader):
        optimizer.zero_grad() # обнуление градиентов оптимизатора
        x_output = model_lstm(x_batch, len_batch) # выход модели для входа x_batch
        loss = criterion(x_output, y_batch) # функция потерь
        loss.backward() # расчёт градиентов
        optimizer.step() # обновление градиентов
        train_loss += loss.item()


    train_loss /= len(train_dataloader)
    val_loss, val_acc = evaluate_single_token(model_lstm, valid_dataloader)
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Val Accuracy: {val_acc:.2%}")

  0%|          | 0/27 [00:20<?, ?it/s]


ValueError: Expected input batch_size (26) to match target batch_size (128).

In [38]:
print(x_output.shape)
print(y_batch.shape)

torch.Size([26, 128, 50257])
torch.Size([128])
